# Dependencies

In [4]:
pip install opencv-python


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Part 0.1: Calibrating Your Camera

In [ ]:
import cv2
import numpy as np
import os

# Create ArUco dictionary and detector parameters (4x4 tags)
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
aruco_params = cv2.aruco.DetectorParameters()

def import_images(path):
    images = []
    for filename in os.listdir(path):
        if filename.lower().endswith('.jpg'):
            img_path = os.path.join(path, filename)
            image = cv2.imread(img_path)
            if image is not None:
                images.append(image)
    return images

def get_points(images):
    object_points = [] # world space 3D
    image_points = [] # pixel space 2D

    single_tag_object_points = np.array([
        [0.0, 0.0, 0.0],
        [0.02, 0.0, 0.0],
        [0.02, 0.02, 0.0],
        [0.0, 0.02, 0.0]
    ], dtype=np.float32)
    image_size = None

    for image in images:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        if image_size is None:
            image_size = gray.shape[::-1] # w, h

        # Detect ArUco markers in an image
        # Returns: corners (list of numpy arrays), ids (numpy array)
        corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)

        # Check if any markers were detected
        if ids is not None:
            # Process the detected corners
            # corners: list of length N (number of detected tags)
            #   - each element is a numpy array of shape (1, 4, 2) containing the 4 corner coordinates (x, y)
            # ids: numpy array of shape (N, 1) containing the tag IDs for each detected marker
            # Example: if 3 tags detected, corners will be a list of 3 arrays, ids will be shape (3, 1)
            obj_points_for_this_image = np.empty((0, 3), dtype=np.float32)
            img_points_for_this_image = np.empty((0, 2), dtype=np.float32)

            for i in range(len(ids)):
                obj_points_for_this_image = np.vstack((obj_points_for_this_image, single_tag_object_points))
                img_points_for_this_image = np.vstack((img_points_for_this_image, corners[i].reshape(4, 2)))
                
            object_points.append(obj_points_for_this_image)
            image_points.append(img_points_for_this_image)
            cv2.aruco.drawDetectedMarkers(image, corners, ids)
            cv2.imshow('Detections', image)
            cv2.waitKey(1500)
        else:
            # No tags detected in this image, skip it
            pass
        
    cv2.destroyAllWindows()
    return object_points, image_points, image_size

images = import_images("./aruco_images")
object_points, image_points, image_size = get_points(images)

## Calibrate

In [ ]:
ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
        object_points, 
        image_points, 
        image_size,  # w, h
        None, 
        None
    )

if ret:
    print("\nCamera Matrix (K):")
    print(camera_matrix)
    
    print("\nDistortion Coefficients:")
    print(dist_coeffs)
    np.savez('./calibrate_results/camera_calibration.npz', camera_matrix=camera_matrix, dist_coeffs=dist_coeffs)
        
else:
    print("\nCalibration Failed")

## Part 0.2: Capturing a 3D Object Scan

Placed in ./object_images

## Part 0.3: Estimating Camera Pose


In [ ]:
import cv2
import numpy as np
import os
import glob
import time

CALIBRATION_DIR = './aruco_images'
OBJECT_SCAN_DIR = './object_images' 
TAG_SIZE_METERS = 0.02
FRUSTUM_SCALE = 0.02
CALIBRATION_FILE = './calibrate_results/camera_calibration.npz'

if os.path.exists(CALIBRATION_FILE):
    print(f"Loading calibration")
    calibration_data = np.load(CALIBRATION_FILE)
    camera_matrix = calibration_data['camera_matrix']
    dist_coeffs = calibration_data['dist_coeffs']
else:
    print("No calibration. need Part 0.1")

object_scan_images = import_images(OBJECT_SCAN_DIR)
if not object_scan_images:
    print("Need Part 0.2")
    exit()

print(f"Starting pose estimation for {len(object_scan_images)} images...")

single_tag_object_points = np.array([
    [0.0, 0.0, 0.0],
    [TAG_SIZE_METERS, 0.0, 0.0],
    [TAG_SIZE_METERS, TAG_SIZE_METERS, 0.0],
    [0.0, TAG_SIZE_METERS, 0.0]
], dtype=np.float32)

valid_c2w_matrices = []
valid_images = []

H, W = 0, 0

for img in object_scan_images:
    if H == 0:
        H, W = img.shape[:2]

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Detect markers
    corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)

    if ids is not None and len(ids) == 1:
        imagePoints = corners[0].reshape(4, 2)
        
        success, rvec, tvec = cv2.solvePnP(
            objectPoints=single_tag_object_points, 
            imagePoints=imagePoints, 
            cameraMatrix=camera_matrix, 
            distCoeffs=dist_coeffs
        )
        
        if success:
            R, _ = cv2.Rodrigues(rvec)
            
            w2c_matrix = np.eye(4)
            w2c_matrix[:3, :3] = R
            w2c_matrix[:3, 3] = tvec.ravel()
            
            c2w_matrix = np.linalg.inv(w2c_matrix)
            
            valid_c2w_matrices.append(c2w_matrix)
            valid_images.append(img)
            
        else:
            print(f"solvePnP failed")
    else:
        print(f"No tag")

print(f"\nPose done. {len(valid_c2w_matrices)} valid poses.")

In [ ]:
import viser
import numpy as np

server = viser.ViserServer(share=True)

focal = camera_matrix[0, 0]
fov_y = 2 * np.arctan2(H / 2, focal)
aspect_ratio = W / H

for i in range(len(valid_c2w_matrices)):
    c2w = valid_c2w_matrices[i]
    img = valid_images[i]
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    server.scene.add_camera_frustum(
        f"/cameras/{i}",
        fov=fov_y,
        aspect=aspect_ratio,
        scale=FRUSTUM_SCALE,
        wxyz=viser.transforms.SO3.from_matrix(c2w[:3, :3]).wxyz,
        position=c2w[:3, 3],
        image=img_rgb
    )

print("Visualization running. Open the URL in your browser.")
while True:
    time.sleep(0.1)

ModuleNotFoundError: No module named 'viser'